# Chat API LLM Detector - Google Colab

Detect which chat API (ChatGPT, Claude, Grok, Gemini, or DeepSeek) generated a piece of text.

**Important:** Make sure to enable GPU runtime!
- Go to: Runtime → Change runtime type → Hardware accelerator → GPU (T4)

## Step 1: Install Dependencies

Installing compatible versions of all required packages...

**Note:** This project uses Llama 3, which requires Hugging Face authentication.

In [ ]:
# Install compatible versions to avoid dependency conflicts
!pip install -q transformers==4.46.3 peft==0.13.2 huggingface-hub accelerate
!pip install -q llm2vec==0.2.3

print("✓ Dependencies installed successfully!")

## Step 3: Download the Classifier

This will download ~16GB. It may take 5-10 minutes.

In [ ]:
from huggingface_hub import login

# Try to get token from Colab secrets
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("✓ Successfully authenticated with Hugging Face!")
except:
    # If secret fails, try manual input
    print("⚠️  Could not find HF_TOKEN in Colab secrets.")
    print("\nOption 1 - Add to Colab Secrets (recommended):")
    print("1. Request access: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct")
    print("2. Create token: https://huggingface.co/settings/tokens")
    print("3. Click the 🔑 icon on the left sidebar")
    print("4. Add a secret named 'HF_TOKEN' with your token")
    print("5. Enable notebook access toggle")
    print("6. Restart runtime and re-run this cell")
    print("\nOption 2 - Manual login (less secure):")
    from getpass import getpass
    token = getpass("Enter your HuggingFace token: ")
    login(token=token)
    print("✓ Authenticated!")

## Step 4: Load the Classifier

In [ ]:
!huggingface-cli download Yida/classifier_chat --local-dir ./classifier_chat

## Step 5: Define Prediction Function

In [ ]:
import torch
import numpy as np
from transformers import AutoConfig, AutoModel, AutoTokenizer
from peft import PeftModel
from llm2vec import LLM2Vec
import gc
import os

def load_classifier(checkpoint_path="./classifier_chat", num_labels=5):
    """Load the pre-trained LLM2Vec classifier with memory optimization."""
    print("Loading classifier with memory optimization...")
    
    # Clear cache first
    torch.cuda.empty_cache()
    gc.collect()
    
    base_model_name = "McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp"
    
    config = AutoConfig.from_pretrained(
        base_model_name,
        trust_remote_code=True,
    )
    
    # Determine device strategy based on available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Detected {num_gpus} GPU(s)")
    
    if num_gpus > 1:
        # Use sequential multi-GPU allocation (not balanced)
        # Sequential puts early layers on GPU 0, later layers on GPU 1
        # This works better than balanced for models with residual connections
        print(f"Using {num_gpus} GPUs with sequential allocation")
        max_memory = {i: "13GiB" for i in range(num_gpus)}
        max_memory["cpu"] = "30GiB"
        device_map = "sequential"  # Sequential instead of balanced
    elif num_gpus == 1:
        # Single GPU with CPU offload if needed
        print("Using single GPU with CPU offload")
        max_memory = {0: "13GiB", "cpu": "30GiB"}
        device_map = "auto"
    else:
        # CPU only
        print("No GPU detected, using CPU")
        max_memory = None
        device_map = "cpu"
    
    # Load base model
    print("Loading base model...")
    model = AutoModel.from_pretrained(
        base_model_name,
        config=config,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
        max_memory=max_memory,
        offload_folder="./offload",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    
    print("✓ Base model loaded")
    torch.cuda.empty_cache()
    
    # Load and merge first adapter
    print("Loading first adapter...")
    model = PeftModel.from_pretrained(
        model,
        base_model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )
    
    # Move to CPU for merging to avoid OOM
    print("✓ Moving to CPU for merging...")
    model = model.cpu()
    torch.cuda.empty_cache()
    gc.collect()
    
    print("✓ Merging first adapter...")
    model = model.merge_and_unload()
    
    # Move back to GPU(s)
    print("✓ Moving merged model back to GPU(s)...")
    model = model.to(torch.bfloat16)
    
    # Re-dispatch to GPU(s) with sequential mapping
    from accelerate import dispatch_model, infer_auto_device_map
    device_map_merged = infer_auto_device_map(
        model,
        max_memory=max_memory,
        no_split_module_classes=["LlamaDecoderLayer"]
    )
    model = dispatch_model(model, device_map=device_map_merged, offload_dir="./offload")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    # Load supervised adapter
    print("✓ Loading supervised adapter...")
    model = PeftModel.from_pretrained(
        model,
        f"{base_model_name}-supervised",
        is_trainable=True,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )
    
    torch.cuda.empty_cache()
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    model = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)
    
    hidden_size = list(model.modules())[-1].weight.shape[0]
    model.head = torch.nn.Linear(hidden_size, num_labels, dtype=torch.bfloat16)
    
    head_path = os.path.join(checkpoint_path, "head.pt")
    
    # Put head on first available device
    if hasattr(model.model, 'hf_device_map'):
        first_device = list(model.model.hf_device_map.values())[0]
        if isinstance(first_device, int):
            device = torch.device(f"cuda:{first_device}")
        else:
            device = torch.device(first_device)
    else:
        device = next(model.parameters()).device
    
    model.head.load_state_dict(torch.load(head_path, map_location=device))
    model.head = model.head.to(device)
    
    model.eval()
    
    # Final cleanup
    torch.cuda.empty_cache()
    gc.collect()
    
    # Show memory usage
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            print(f"✓ GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    else:
        print("✓ Classifier loaded on CPU")
    
    return model

# Load the model
model = load_classifier()

## Step 6: Test with Sample Text

Replace the text below with any text you want to classify:

In [ ]:
def predict_text(model, text):
    """Predict which LLM generated the given text."""
    label_names = ["ChatGPT", "Claude", "Grok", "Gemini", "DeepSeek"]
    
    # Prepare text
    prepared_text = model.prepare_for_tokenization(text)
    inputs = model.tokenize([prepared_text])
    
    # IMPORTANT: For multi-GPU setups, always put inputs on cuda:0
    # The model will handle moving tensors between GPUs automatically
    if torch.cuda.is_available():
        target_device = torch.device("cuda:0")
    else:
        target_device = next(model.parameters()).device
    
    inputs = {k: v.to(target_device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        embeddings = model.forward(inputs)
        
        # Move embeddings to same device as classification head
        if hasattr(model, 'head'):
            head_device = next(model.head.parameters()).device
            embeddings = embeddings.to(head_device)
        
        embeddings = embeddings.to(torch.bfloat16)
        logits = model.head(embeddings)
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    pred_label = torch.argmax(probabilities, dim=-1).item()
    all_probs = probabilities[0].cpu().numpy()
    
    # Display results
    print("\n" + "="*60)
    print("PREDICTION RESULTS")
    print("="*60)
    print(f"\nMost likely source: {label_names[pred_label]}")
    print(f"Confidence: {all_probs[pred_label]*100:.2f}%")
    print("\nAll probabilities:")
    print("-"*60)
    
    sorted_indices = np.argsort(all_probs)[::-1]
    for idx in sorted_indices:
        bar_length = int(all_probs[idx] * 50)
        bar = "█" * bar_length
        print(f"{label_names[idx]:20s} {all_probs[idx]*100:6.2f}% {bar}")
    print("="*60)
    
    return label_names[pred_label], all_probs[pred_label]

## Step 7: Classify Your Own Text

Paste your text in the input box below:

In [ ]:
# Example text - replace with your own!
sample_text = """
Hello! I'd be happy to help you with that question. Let me break this down into a few key points:

1. First, it's important to understand the context
2. Second, we should consider the implications
3. Finally, let's look at practical applications

I hope this helps clarify things for you!
"""

predict_text(model, sample_text)

## Step 6: Classify Your Own Text

Paste your text in the input box below:

In [ ]:
# Interactive input
print("Paste your text below and press Enter:")
user_text = input()

if user_text.strip():
    predict_text(model, user_text)
else:
    print("No text provided!")

## Multiple Texts

You can test multiple texts at once:

In [ ]:
# Test multiple texts
texts_to_test = [
    "Sure, I can help with that!",
    "I'd be happy to assist you with this question.",
    "Let me break this down for you step by step.",
]

for i, text in enumerate(texts_to_test, 1):
    print(f"\n{'='*60}")
    print(f"TEXT #{i}: {text[:50]}...")
    predict_text(model, text)